# Approach 2.5 / 3.5

Each of the approaches tested so far, despite being used previously in the literature (at least to the extent that I understand them) have failed to out perform just randomly selecting samples. I suspect this could be due to one and/or another issue with the current acquisition strategy. Each of these approaches use some method to find the variance of model predictions to find the most uncertain sequences for the next batch. However, those uncertain sequences might somehow correspond to noisy measurements (measurements with lower read counts, especially if they tend to be the sequences with more mutations), and they might not be sufficiently diverse.

Given these hypotheses for what's going wrong, I may be able to solve them by ensuring the next sample is uncertain as well as diverse. I'd be interested in testing when its just diverse and when its uncertain and diverse compared to the random baseline.

But how do we ensure diversity? Well, since we are conveniently using a language model, we should be able to just use the language model embeddings. So, all I need to do is figure out how to get the embeddings, measure their distance, and integrate some amount of distance optimization into my acquistion function.

Note: Below, I'm going to extract the embeddings from the last hidden state of the CLS token, since that's the standard model I've been working with setting up the structure of active learning. However, other fine-tuning approaches should make use of different embeddings. For example, if you're using only the mutant residues as the final state input to your output layer, you should probably use those embeddings for diversity. Another decision point is whether to use the embeddings of your fine-tuned model or a fresh ESM2 model. I'm going to work with just the fine-tuned model here.

Here's the algorithm I'll employ:
1. Train ensemble on random initial set.
2. Get variances from model predictions.
3. Filter samples with top k% of variances.
4. Retrieve embeddings for those samples.
5. For each sample in the unlabeled pool, calculate its distance from each of the samples in the current batch.
6. Find the minimum distance from those for each candidate.
7. Add candidate c whose minimum distance from the current batch is the greatest and remove it from the unlabeled pool.
8. Calculate the distance of just-added candidate c from each of the remaining unlabeled pool. Add these distances to the distances list for each sample.
9. Repeat 6-8 until n samples are added to the labeled samples.

Actually, now that I spell out the algorithm, I'm realizing that I should also test just a random choice from the top k% of variances to make sure that the embeddings even add anything.

In [1]:
from scripts.data_utils import train_val_test_split
from scripts.config import (
    DATA_PATH, 
    SEQUENCE_COL, 
    SCORE_COL, 
    TOK_MODEL, 
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED,
)

training_pool, val_dataloader, test_dataloader = train_val_test_split(
    DATA_PATH,
    SEQUENCE_COL,
    SCORE_COL,
    TOK_MODEL,
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED
)

In [2]:
import torch
from torch.utils.data import Subset, DataLoader
import numpy as np

from scripts.training import initialize_and_train_new_model
from scripts.acquisition import get_pool_predictions

def get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size):
    bootstrap_indices = np.random.choice(labeled_indices, size=int(0.9*len(labeled_indices)),replace=True)
    bootstrap_subset = Subset(pool_dataset, bootstrap_indices)
    bootstrap_dataloader = DataLoader(bootstrap_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    return bootstrap_dataloader

def train_ensemble(
        n_models, 
        model_name, 
        approach,
        learning_rate,
        weight_decay,
        epochs,
        labeled_indices,
        train_dataloader_batch_size,
        pool_dataset, 
        pool_dataloader, 
        val_dataloader,
        patience
        ):
    
    # define list to store predictions as each model is trained then evaluated
    ensemble_predictions = []
    
    for i in range(n_models):
        print(f"\nTraining Model {i+1}...")
        # set a changing manual seed
        torch.manual_seed(i)
        torch.cuda.manual_seed(i)

        # get bootstrap sample from labeled dataset
        bootstrap_dataloader = get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size)

        # initialize and train a new model
        model = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, bootstrap_dataloader, val_dataloader, patience)
        
        # get model predictions on pool dataloader, append to ensemble predictions list
        pool_preds = get_pool_predictions(model, pool_dataloader)
        ensemble_predictions.append(pool_preds)

    # stack ensemble predictions to create tensor of shape (n_models, n_unlabeled_samples)
    ensemble_predictions = torch.stack(ensemble_predictions, dim=0)
    print("Ensemble training complete, submitting predictions for next cycle.")
    # return list of ensemble predictions
    return ensemble_predictions

### Random choice from top % variances  

In [9]:
import numpy as np
from torch.utils.data import Subset, DataLoader

# acquire new batch, randomly if no scores given, top "batch_size_to_acquire" if given
def acquire_new_batch(
    dataset, 
    train_dataloader_batch_size, 
    pool_dataloader_batch_size, 
    initial_batch_size,
    top_score_fraction, 
    batch_size_to_acquire, 
    labeled_indices, 
    unlabeled_indices, 
    acquisition_scores=None
    ):

    # if initial batch, when there are no acquisition scores, select randomly
    if acquisition_scores is None:
        initial_batch_size = min(initial_batch_size, len(unlabeled_indices))
        indices_to_acquire = np.random.choice(unlabeled_indices, size=initial_batch_size, replace=False)
    
    # else select based on top acquisition scores
    else:
        # make sure we don't overshoot samples to acquire if on the final batch
        batch_size_to_acquire = min(batch_size_to_acquire, len(acquisition_scores))
        # detmine the number of top scorers to select from
        num_top_scorers = int(top_score_fraction * len(unlabeled_indices))
        # get the indicies of the top acquisition scores (num of samples)
        top_indices = acquisition_scores.topk(num_top_scorers).indices
        # choose a random set of indices from these top scorers
        top_k_indices = np.random.choice(top_indices.cpu().numpy(), size=batch_size_to_acquire, replace=False)
        # use these to find the indices that map back to the original dataset
        indices_to_acquire = unlabeled_indices[top_k_indices]
    
    # update the indices lists
    labeled_indices = np.concatenate([labeled_indices, indices_to_acquire])
    unlabeled_indices = np.setdiff1d(unlabeled_indices, indices_to_acquire, assume_unique=True)
    
    # create new subsets and dataloaders
    train_subset = Subset(dataset, labeled_indices.tolist())
    pool_subset = Subset(dataset, unlabeled_indices.tolist())
    train_dataloader = DataLoader(train_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    pool_dataloader = DataLoader(pool_subset, batch_size=pool_dataloader_batch_size, shuffle=False)
    
    return train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices

In [10]:
from pathlib import Path
import pandas as pd

from scripts.acquisition import get_variances
from scripts.training import initialize_and_train_new_model, test_model
from scripts.campaigns import run_standard_finetuning


def get_learning_curves(
        n_samples,
        initial_n_samples,
        top_score_fraction,
        n_samples_per_batch,
        model_name, 
        approach,
        learning_rate, 
        weight_decay, 
        epochs, 
        training_pool, 
        train_dataloader_batch_size,
        pool_dataloader_batch_size,
        val_dataloader, 
        test_dataloader,
        patience=5,
        n_models=5,
        results_path="active_vs_standard_learning_curves.csv"
):
    results_path = Path(results_path)
    results_dir = results_path.parent
    results_dir.mkdir(parents=True, exist_ok=True)

    # Load existing results if the file exists, otherwise start with a fresh DataFrame.
    if results_path.exists():
        all_results_df = pd.read_csv(results_path)
    else:
        all_results_df = pd.DataFrame()
    
    total_pool_size = len(training_pool)
    unlabeled_indices = np.arange(total_pool_size)
    labeled_indices = np.array([], dtype=np.int64)

    ensemble_predictions = None
    current_cycle = 1
    total_cycles = int(np.ceil((n_samples-initial_n_samples)/n_samples_per_batch)) + 1
    
    while len(labeled_indices) < n_samples and len(unlabeled_indices) > 0:
        print(f"\nCycle {current_cycle}/{total_cycles}\n-------------------------------------------------")

        # on the first cycle, choose random samples of initial_n_samples size
        if ensemble_predictions is None:
            print(f"Choosing initial {initial_n_samples} samples randomly...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, top_score_fraction, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=None
            )
        # each other time, use the n_samples_per_batch with acquisition scores to select
        else:
            scores = get_variances(ensemble_predictions, f"{results_dir}/variances{current_cycle}.csv")
            print(f"Selecting new data points...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, top_score_fraction, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=scores
            )
        
        # give message when loop ends
        if len(unlabeled_indices) == 0:
            print("Unlabeled pool is empty. Proceeding to final model training.")
            break
        
        # evaluate active vs standard
        final_results = []

        # active
        print(f"\nTraining and evaluating model using {len(labeled_indices)} actively selected samples...")
        model_active = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, train_dataloader, val_dataloader, patience, return_history=False)
        results_active = test_model(model_active, test_dataloader, return_results=True)
        results_active = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'active',
            **results_active
        }
        final_results.append(results_active)

        # standard
        print(f"\nTraining and evaluating model using {len(labeled_indices)} randomly selected samples...")
        model_standard, _ = run_standard_finetuning(len(labeled_indices), approach, model_name, train_dataloader_batch_size, learning_rate, weight_decay, epochs, training_pool, val_dataloader, patience)
        results_standard = test_model(model_standard, test_dataloader, return_results=True)
        results_standard = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'standard',
            **results_standard
        }
        final_results.append(results_standard)
        # save to disk each time to save progress
        results_df = pd.DataFrame(final_results)
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
        all_results_df.to_csv(results_path, index=False)
        print(f"Progress for experiment {current_cycle-1} appended to {results_path}")

        # if it's the last cycle, skip ensemble predictions
        if (current_cycle == total_cycles):
            print("Experiments complete.")
            break

        print("Starting ensemble training and pool evaluation...")
        ensemble_predictions = train_ensemble(n_models, model_name, approach, learning_rate, weight_decay, epochs, labeled_indices, train_dataloader_batch_size, training_pool, pool_dataloader, val_dataloader, patience)
    
        current_cycle += 1
    return all_results_df

In [11]:
from scripts.config import (
    MODEL_NAME,
    APPROACH,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EPOCHS,
    POOL_BATCH_SIZE,
    PATIENCE,
    N_MODELS,
    TOP_SCORE_FRACTION,
)

get_learning_curves(
    n_samples=256,
    initial_n_samples=16,
    top_score_fraction=TOP_SCORE_FRACTION,
    n_samples_per_batch=16,
    model_name=MODEL_NAME,
    approach=APPROACH,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    training_pool=training_pool,
    train_dataloader_batch_size=BATCH_SIZE,
    pool_dataloader_batch_size=POOL_BATCH_SIZE,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    patience=PATIENCE,
    n_models=N_MODELS,
    results_path='results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv'
)


Cycle 1/16
-------------------------------------------------
Choosing initial 16 samples randomly...

Training and evaluating model using 16 actively selected samples...


/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Training]: 100%|██████████| 50/50 [00:35<00:00,  1.42it/s]


Train Loss: 0.0336 | Val Loss: 0.1788 | SpearmanR: 0.4060


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 127.40it/s]



Training and evaluating model using 16 randomly selected samples...


...

[Training]: 100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


Train Loss: 0.0382 | Val Loss: 0.1242 | SpearmanR: 0.6988


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 118.27it/s]

Progress for experiment 15 appended to results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv
Experiments complete.


,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.232308,0.276791,0.255876,0.232170
1,n_samples,0,16,standard,0.208022,0.108390,0.151105,0.207960
2,n_samples,0,16,active,0.225947,0.288939,0.327616,0.226419
3,n_samples,0,16,standard,0.223339,-0.140400,-0.167712,0.223238
4,n_samples,1,32,active,0.184960,0.453173,0.476212,0.185616
5,n_samples,1,32,standard,0.213183,0.243198,0.218171,0.213262
6,n_samples,2,48,active,0.164185,0.492976,0.524374,0.164829
7,n_samples,2,48,standard,0.218176,0.484964,0.408690,0.218421
8,n_samples,3,64,active,0.152050,0.509348,0.531544,0.152574
9,n_samples,3,64,standard,0.144240,0.547185,0.551789,0.143950


In [12]:
lc_df = pd.read_csv("results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv")
lc_df.head()

,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.225947,0.288939,0.327616,0.226419
1,n_samples,0,16,standard,0.223339,-0.140400,-0.167712,0.223238
2,n_samples,1,32,active,0.184960,0.453173,0.476212,0.185616
3,n_samples,1,32,standard,0.213183,0.243198,0.218171,0.213262
4,n_samples,2,48,active,0.164185,0.492976,0.524374,0.164829


In [13]:
import plotly.express as px

fig = px.line(lc_df, 'value', 'spearmanr',color="training_method")

fig.update_layout(
    xaxis_title="Number of Training Samples",
    yaxis_title="Spearman Correlation Coefficient"
)

fig.show()

In [14]:
import pandas as pd
import glob
from pathlib import Path

file_pattern = "results/04_adding_diversity/random_from_top_fraction/variances*.csv"
variance_files = glob.glob(file_pattern)
variance_files.sort()

all_variances_dfs = []

for filepath in variance_files:
    temp_df = pd.read_csv(filepath)
    column_name = Path(filepath).stem
    temp_df = temp_df.rename(columns={'variance': column_name})
    all_variances_dfs.append(temp_df)

final_df = pd.concat(all_variances_dfs, axis=1)

final_df

,variances10,variances11,variances12,variances13,variances14,variances15,variances16,variances2,variances3,variances4,variances5,variances6,variances7,variances8,variances9
0,0.017557,0.017802,0.021863,0.012007,0.007614,0.003172,0.006799,0.021065,0.034405,0.087627,0.007492,0.010298,0.001138,0.001611,0.073645
1,0.030268,0.013252,0.017392,0.014582,0.000798,0.013131,0.012300,0.032668,0.025599,0.013012,0.006858,0.012764,0.006510,0.019254,0.046434
2,0.062635,0.009125,0.005722,0.016405,0.020386,0.018690,0.008005,0.180021,0.043308,0.125256,0.021468,0.057188,0.073069,0.013695,0.049970
3,0.006008,0.047597,0.165927,0.025047,0.017256,0.031446,0.033810,0.034345,0.010135,0.010568,0.011698,0.011238,0.002768,0.009420,0.062151
4,0.108935,0.037727,0.017766,0.017103,0.021608,0.025763,0.023030,0.034272,0.086913,0.054073,0.031779,0.078399,0.056023,0.042748,0.117974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023604,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064340,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022443,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
columns_to_plot = ['variances2', 'variances5', 'variances8', 'variances11', 'variances14']

# "Melt" the selected columns into a long-format DataFrame
melted_df = final_df[columns_to_plot].melt(
    var_name='Cycle',      # New column for the original column names
    value_name='Variance'  # New column for the variance values
)

fig = px.histogram(
    data_frame=melted_df,
    x='Variance',                                  
    color='Cycle',                                 
    barmode='overlay',                             
    opacity=0.65,                                  
    histnorm='probability density',                
    title='Distribution of Variances Across Active Learning Cycles'
)

fig.show()

### With embeddings

#### Variance + Embedding Diversity

In [3]:
def get_embeddings(dataset, indices, pool_dataloader_batch_size, model):
    device = 'cuda' if torch.cuda.is_available else 'cpu'
    
    # Use input indices to make a subset and dataloader from the dataset.
    candidate_subset = Subset(dataset, indices)
    candidate_dataloader = DataLoader(candidate_subset, batch_size=pool_dataloader_batch_size, shuffle=False)
    # initialize a list to store embeddings
    embeddings = []
    # Loop through the dataloader
    model.eval()
    with torch.inference_mode():
        for inputs, _ in candidate_dataloader:
            # forward pass on the batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs, output_hidden_states=True)
            # get last hidden states
            last_hidden_states = outputs.hidden_states[-1]
            # get the embeddings of the first token (CLS token)
            embedding = last_hidden_states[:,0,:]
            # append to embedding list
            embeddings.append(embedding)
    # return embeddings, converted to tensor, without batch dimension
    embeddings = torch.cat(embeddings, dim=0)
    return embeddings

def select_diverse_batch(labeled_embeddings, candidate_embeddings, n_samples_per_batch):
    # ensure we don't try to get more candidates than we have available
    num_candidates = candidate_embeddings.shape[0]
    n_samples_in_batch = min(n_samples_per_batch, num_candidates)

    # initialize list to store selected indices
    selected_indices = []

    # get initial euclidean distances, final shape: # candidate samples x # labeled samples
    dist_matrix = torch.cdist(candidate_embeddings, labeled_embeddings, p=2)
    
    # find the minimum distance each candidate sample is from any labeled sample
    min_distances, _ = torch.min(dist_matrix, dim=1) 

    # for each sample in new batch
    for _ in range(n_samples_in_batch):
        # find the index of the sample with the greatest distance
        farthest_idx = torch.argmax(min_distances).item()
        # add that index to the selected indices list
        selected_indices.append(farthest_idx)
        # get the embedding of the newly selected sample
        newly_selected_embedding = candidate_embeddings[farthest_idx].unsqueeze(0) # unsqueeze to add batch dimension for cdist to function
        # calculate the new distances from the newly selected sample
        dist_to_new = torch.cdist(candidate_embeddings, newly_selected_embedding, p=2).squeeze()
        # update minimum distances for the next iteration
        min_distances = torch.minimum(min_distances, dist_to_new)
    
    return selected_indices

def acquire_new_batch(
        dataset, 
        train_dataloader_batch_size, 
        pool_dataloader_batch_size, 
        initial_batch_size, 
        top_score_fraction,
        batch_size_to_acquire, 
        labeled_indices, 
        unlabeled_indices,
        model=None, 
        variances=None,
        ):

    # if initial batch, when there are no variances, select randomly
    if variances is None:
        initial_batch_size = min(initial_batch_size, len(unlabeled_indices))
        indices_to_acquire = np.random.choice(unlabeled_indices, size=initial_batch_size, replace=False)
    
    # else select based on top variances scores
    else:
        # make sure we don't overshoot samples to acquire if on the final batch
        batch_size_to_acquire = min(batch_size_to_acquire, len(variances))
        # determine the number of top scorers to select from
        num_top_scorers = int(top_score_fraction * len(unlabeled_indices))
        # get the indicies of the top acquisition scores (num of samples)
        candidate_indices = variances.topk(num_top_scorers).indices
        
        # get embeddings for labeled and candidate sets
        labeled_embeds = get_embeddings(dataset, labeled_indices, pool_dataloader_batch_size, model)
        candidate_embeds = get_embeddings(dataset, candidate_indices, pool_dataloader_batch_size, model)

        # use embeddings to determine candidate indices that are diverse
        diverse_indices_in_candidates = select_diverse_batch(labeled_embeds, candidate_embeds, batch_size_to_acquire)

        # use these to find the indices that map back to the original dataset
        indices_to_acquire = candidate_indices[diverse_indices_in_candidates]
    
    # update the indices lists
    labeled_indices = np.concatenate([labeled_indices, indices_to_acquire])
    unlabeled_indices = np.setdiff1d(unlabeled_indices, indices_to_acquire, assume_unique=True)
    
    # create new subsets and dataloaders
    train_subset = Subset(dataset, labeled_indices.tolist())
    pool_subset = Subset(dataset, unlabeled_indices.tolist())
    train_dataloader = DataLoader(train_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    pool_dataloader = DataLoader(pool_subset, batch_size=pool_dataloader_batch_size, shuffle=False)
    
    return train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices

In [4]:
from pathlib import Path
import pandas as pd

from scripts.acquisition import get_variances
from scripts.training import initialize_and_train_new_model, test_model
from scripts.campaigns import run_standard_finetuning


def get_learning_curves(
        n_samples,
        initial_n_samples,
        top_score_fraction,
        n_samples_per_batch,
        model_name, 
        approach,
        learning_rate, 
        weight_decay, 
        epochs, 
        training_pool, 
        train_dataloader_batch_size,
        pool_dataloader_batch_size,
        val_dataloader, 
        test_dataloader,
        patience=5,
        n_models=5,
        results_path="active_vs_standard_learning_curves.csv"
):
    results_path = Path(results_path)
    results_dir = results_path.parent
    results_dir.mkdir(parents=True, exist_ok=True)

    # Load existing results if the file exists, otherwise start with a fresh DataFrame.
    if results_path.exists():
        all_results_df = pd.read_csv(results_path)
    else:
        all_results_df = pd.DataFrame()
    
    total_pool_size = len(training_pool)
    unlabeled_indices = np.arange(total_pool_size)
    labeled_indices = np.array([], dtype=np.int64)

    ensemble_predictions = None
    current_cycle = 1
    total_cycles = int(np.ceil((n_samples-initial_n_samples)/n_samples_per_batch)) + 1
    
    while len(labeled_indices) < n_samples and len(unlabeled_indices) > 0:
        print(f"\nCycle {current_cycle}/{total_cycles}\n-------------------------------------------------")

        # on the first cycle, choose random samples of initial_n_samples size
        if ensemble_predictions is None:
            print(f"Choosing initial {initial_n_samples} samples randomly...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, 
                train_dataloader_batch_size, 
                pool_dataloader_batch_size, 
                initial_n_samples, 
                top_score_fraction, 
                n_samples_per_batch, 
                labeled_indices, 
                unlabeled_indices, 
                variances=None
            )
        # each other time, use the n_samples_per_batch with acquisition scores to select
        else:
            variances = get_variances(ensemble_predictions, f"{results_dir}/variances{current_cycle}.csv")
            print(f"Selecting new data points...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, 
                train_dataloader_batch_size, 
                pool_dataloader_batch_size, 
                initial_n_samples, 
                top_score_fraction, 
                n_samples_per_batch, 
                labeled_indices, 
                unlabeled_indices,
                model=model_active,
                variances=variances
            )
        
        # give message when loop ends
        if len(unlabeled_indices) == 0:
            print("Unlabeled pool is empty. Proceeding to final model training.")
            break
        
        # evaluate active vs standard
        final_results = []

        # active
        print(f"\nTraining and evaluating model using {len(labeled_indices)} actively selected samples...")
        model_active = initialize_and_train_new_model(
            approach, 
            model_name, 
            learning_rate, 
            weight_decay, 
            epochs, 
            train_dataloader, 
            val_dataloader, 
            patience, 
            return_history=False
            )
        results_active = test_model(model_active, test_dataloader, return_results=True)
        results_active = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'active',
            **results_active
        }
        final_results.append(results_active)

        # standard
        print(f"\nTraining and evaluating model using {len(labeled_indices)} randomly selected samples...")
        model_standard, _ = run_standard_finetuning(
            len(labeled_indices), 
            approach, 
            model_name, 
            train_dataloader_batch_size, 
            learning_rate, 
            weight_decay, 
            epochs, 
            training_pool, 
            val_dataloader, 
            patience
            )
        results_standard = test_model(model_standard, test_dataloader, return_results=True)
        results_standard = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'standard',
            **results_standard
        }
        final_results.append(results_standard)
        # save to disk each time to save progress
        results_df = pd.DataFrame(final_results)
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
        all_results_df.to_csv(results_path, index=False)
        print(f"Progress for experiment {current_cycle-1} appended to {results_path}")

        # if it's the last cycle, skip ensemble predictions
        if (current_cycle == total_cycles):
            print("Experiments complete.")
            break

        print("Starting ensemble training and pool evaluation...")
        ensemble_predictions = train_ensemble(
            n_models, 
            model_name, 
            approach, 
            learning_rate, 
            weight_decay, 
            epochs, 
            labeled_indices, 
            train_dataloader_batch_size, 
            training_pool, 
            pool_dataloader, 
            val_dataloader, 
            patience
            )
    
        current_cycle += 1
    return all_results_df

In [ ]:
from scripts.config import (
    MODEL_NAME,
    APPROACH,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EPOCHS,
    POOL_BATCH_SIZE,
    PATIENCE,
    N_MODELS,
    TOP_SCORE_FRACTION, # 0.5 in this trial
)

get_learning_curves(
    n_samples=256,
    initial_n_samples=16,
    top_score_fraction=TOP_SCORE_FRACTION,
    n_samples_per_batch=16,
    model_name=MODEL_NAME,
    approach=APPROACH,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    training_pool=training_pool,
    train_dataloader_batch_size=BATCH_SIZE,
    pool_dataloader_batch_size=POOL_BATCH_SIZE,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    patience=PATIENCE,
    n_models=N_MODELS,
    results_path='results/04_adding_diversity/diverse_from_top_fraction/active_vs_standard_learning_curve.csv'
)


Cycle 1/16
-------------------------------------------------
Choosing initial 16 samples randomly...

Training and evaluating model using 16 actively selected samples...


/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Training]:  44%|████▍     | 22/50 [00:06<00:08,  3.47it/s]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0805 | Val Loss: 0.2110 | SpearmanR: 0.0976


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 130.48it/s]



Training and evaluating model using 16 randomly selected samples...


...

[Training]:  76%|███████▌  | 38/50 [00:22<00:07,  1.65it/s]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0165 | Val Loss: 0.1162 | SpearmanR: 0.7121


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 127.05it/s]


Progress for experiment 15 appended to results/04_adding_diversity/diverse_from_top_fraction/active_vs_standard_learning_curve.csv
Experiments complete.


,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.194070,0.198219,0.211677,0.194043
1,n_samples,0,16,standard,0.202006,0.302308,0.319394,0.202516
2,n_samples,1,32,active,0.135687,0.523961,0.562829,0.135807
3,n_samples,1,32,standard,0.202622,0.103285,0.104650,0.202585
4,n_samples,2,48,active,0.140236,0.532741,0.558240,0.140727
5,n_samples,2,48,standard,0.165474,0.458487,0.462352,0.165602
6,n_samples,3,64,active,0.178242,0.520566,0.559668,0.178790
7,n_samples,3,64,standard,0.169220,0.516508,0.532650,0.170034
8,n_samples,4,80,active,0.156842,0.559239,0.563365,0.157612
9,n_samples,4,80,standard,0.184357,0.431315,0.472125,0.184751


In [10]:
lc_df = pd.read_csv("results/04_adding_diversity/diverse_from_top_fraction/active_vs_standard_learning_curve.csv")
lc_df.head()

,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.194070,0.198219,0.211677,0.194043
1,n_samples,0,16,standard,0.202006,0.302308,0.319394,0.202516
2,n_samples,1,32,active,0.135687,0.523961,0.562829,0.135807
3,n_samples,1,32,standard,0.202622,0.103285,0.104650,0.202585
4,n_samples,2,48,active,0.140236,0.532741,0.558240,0.140727


In [11]:
import plotly.express as px

fig = px.line(lc_df, 'value', 'spearmanr',color="training_method")

fig.update_layout(
    xaxis_title="Number of Training Samples",
    yaxis_title="Spearman Correlation Coefficient"
)

fig.show()

In [12]:
import pandas as pd
import glob
from pathlib import Path

file_pattern = "results/04_adding_diversity/diverse_from_top_fraction/variances*.csv"
variance_files = glob.glob(file_pattern)
variance_files.sort()

all_variances_dfs = []

for filepath in variance_files:
    temp_df = pd.read_csv(filepath)
    column_name = Path(filepath).stem
    temp_df = temp_df.rename(columns={'variance': column_name})
    all_variances_dfs.append(temp_df)

final_df = pd.concat(all_variances_dfs, axis=1)

final_df

,variances10,variances11,variances12,variances13,variances14,variances15,variances16,variances2,variances3,variances4,variances5,variances6,variances7,variances8,variances9
0,0.017610,0.046133,0.016313,0.003964,0.070412,0.015990,0.066768,0.007045,0.018246,0.082885,0.042747,0.028918,0.031012,0.010689,0.084596
1,0.033090,0.054998,0.037995,0.011245,0.034509,0.015215,0.023972,0.008230,0.022088,0.017902,0.042081,0.055104,0.023352,0.011264,0.017188
2,0.028040,0.005810,0.015404,0.010296,0.007105,0.007289,0.010908,0.041063,0.052175,0.007564,0.010557,0.017585,0.011127,0.012925,0.015223
3,0.049954,0.107769,0.030878,0.018527,0.015323,0.047191,0.021950,0.009003,0.017396,0.031625,0.030911,0.019158,0.011881,0.026429,0.075614
4,0.053315,0.015243,0.061432,0.058543,0.019040,0.016268,0.030274,0.009676,0.024557,0.061230,0.054883,0.063969,0.025561,0.097359,0.017862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013406,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042970,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013507,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007618,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
columns_to_plot = ['variances2', 'variances5', 'variances8', 'variances11', 'variances14']

# "Melt" the selected columns into a long-format DataFrame
melted_df = final_df[columns_to_plot].melt(
    var_name='Cycle',      # New column for the original column names
    value_name='Variance'  # New column for the variance values
)

fig = px.histogram(
    data_frame=melted_df,
    x='Variance',                                  
    color='Cycle',                                 
    barmode='overlay',                             
    opacity=0.65,                                  
    histnorm='probability density',                
    title='Distribution of Variances Across Active Learning Cycles'
)

fig.show()

Seems like introducing additional diversity in the selected samples didn't quite cut it. It seemed to perform better in this one run at the beginning, but the random selection caught up to it eventually and continued to perform better. This means that diversity wasn't really the core issue. The most promising next thing I can try is to see in what way the actively selected samples might be biased. I'm quite curious to see if the samples they happen to be selecting are ones that have lower read counts.

Before that, however, I am interested to test whether my diversity algorithm alone can improve performance if it's choosing from the whole dataset.

#### Embedding Diversity only

In [5]:
from scripts.config import (
    MODEL_NAME,
    APPROACH,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EPOCHS,
    POOL_BATCH_SIZE,
    PATIENCE,
    N_MODELS,
    TOP_SCORE_FRACTION, # 1.0 in this trial to eliminate variance based selection
)

get_learning_curves(
    n_samples=256,
    initial_n_samples=16,
    top_score_fraction=TOP_SCORE_FRACTION,
    n_samples_per_batch=16,
    model_name=MODEL_NAME,
    approach=APPROACH,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    training_pool=training_pool,
    train_dataloader_batch_size=BATCH_SIZE,
    pool_dataloader_batch_size=POOL_BATCH_SIZE,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    patience=PATIENCE,
    n_models=N_MODELS,
    results_path='results/04_adding_diversity/diverse_only/active_vs_standard_learning_curve.csv'
)


Cycle 1/16
-------------------------------------------------
Choosing initial 16 samples randomly...

Training and evaluating model using 16 actively selected samples...


/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Training]:  48%|████▊     | 24/50 [00:09<00:10,  2.50it/s]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0351 | Val Loss: 0.2232 | SpearmanR: 0.3042


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 128.19it/s]



Training and evaluating model using 16 randomly selected samples...


...

[Training]: 100%|██████████| 50/50 [00:29<00:00,  1.68it/s]


Train Loss: 0.0142 | Val Loss: 0.1036 | SpearmanR: 0.7047


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 129.88it/s]


Progress for experiment 15 appended to results/04_adding_diversity/diverse_only/active_vs_standard_learning_curve.csv
Experiments complete.


,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.211525,0.401156,0.401702,0.211477
1,n_samples,0,16,standard,0.204896,0.114329,0.102567,0.204831
2,n_samples,1,32,active,0.185638,0.395251,0.416651,0.185663
3,n_samples,1,32,standard,0.197912,0.416348,0.353456,0.197689
4,n_samples,2,48,active,0.172172,0.394972,0.418391,0.172755
5,n_samples,2,48,standard,0.160614,0.498919,0.502520,0.160458
6,n_samples,3,64,active,0.154525,0.449333,0.484009,0.154607
7,n_samples,3,64,standard,0.166977,0.538453,0.553851,0.167662
8,n_samples,4,80,active,0.161583,0.443069,0.486969,0.161741
9,n_samples,4,80,standard,0.192550,0.422612,0.397876,0.192571


In [6]:
lc_df = pd.read_csv("results/04_adding_diversity/diverse_only/active_vs_standard_learning_curve.csv")
lc_df.head()

,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.211525,0.401156,0.401702,0.211477
1,n_samples,0,16,standard,0.204896,0.114329,0.102567,0.204831
2,n_samples,1,32,active,0.185638,0.395251,0.416651,0.185663
3,n_samples,1,32,standard,0.197912,0.416348,0.353456,0.197689
4,n_samples,2,48,active,0.172172,0.394972,0.418391,0.172755


In [7]:
import plotly.express as px

fig = px.line(lc_df, 'value', 'spearmanr',color="training_method")

fig.update_layout(
    xaxis_title="Number of Training Samples",
    yaxis_title="Spearman Correlation Coefficient"
)

fig.show()

In [10]:
import pandas as pd
import glob
from pathlib import Path

file_pattern = "results/04_adding_diversity/diverse_only/variances*.csv"
variance_files = glob.glob(file_pattern)
variance_files.sort()

all_variances_dfs = []

for filepath in variance_files:
    temp_df = pd.read_csv(filepath)
    column_name = Path(filepath).stem
    temp_df = temp_df.rename(columns={'variance': column_name})
    all_variances_dfs.append(temp_df)

final_df = pd.concat(all_variances_dfs, axis=1)

final_df

,variances10,variances11,variances12,variances13,variances14,variances15,variances16,variances2,variances3,variances4,variances5,variances6,variances7,variances8,variances9
0,0.014805,0.028934,0.034272,0.012778,0.003969,0.010099,0.017359,0.023679,0.162896,0.027817,0.015012,0.009623,0.035736,0.065515,0.071914
1,0.005018,0.028932,0.006239,0.016855,0.017220,0.032834,0.037370,0.020564,0.066002,0.022790,0.007513,0.011224,0.119791,0.043675,0.046227
2,0.045094,0.026108,0.011489,0.001493,0.018163,0.016263,0.020630,0.011918,0.069417,0.013939,0.017774,0.008544,0.113017,0.012292,0.073253
3,0.049536,0.014517,0.024133,0.020011,0.068018,0.013868,0.034904,0.031653,0.054439,0.012073,0.011087,0.025785,0.059592,0.019269,0.019120
4,0.096622,0.040313,0.027719,0.007303,0.019241,0.076452,0.008248,0.034136,0.063569,0.039882,0.011267,0.054363,0.181382,0.031879,0.055138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.020339,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005586,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028586,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
columns_to_plot = ['variances2', 'variances5', 'variances8', 'variances11', 'variances14']

# "Melt" the selected columns into a long-format DataFrame
melted_df = final_df[columns_to_plot].melt(
    var_name='Cycle',      # New column for the original column names
    value_name='Variance'  # New column for the variance values
)

fig = px.histogram(
    data_frame=melted_df,
    x='Variance',                                  
    color='Cycle',                                 
    barmode='overlay',                             
    opacity=0.65,                                  
    histnorm='probability density',                
    title='Distribution of Variances Across Active Learning Cycles'
)

fig.show()

Much of the same conclusions here. Even when you are only choosing based on diversity, there's either a reduction or comparable performance compared to a standard run, suggesting a bias away from the most informative samples, rather than a bias toward the most informative samples. As mentioned before, the next step is to investigate the impact of read counts. Perhaps each of these schemes are biasing toward samples that have lower read counts. 

There's two ways I can get at this question that I can think of right now. One is to map the chosen samples back to the read counts and see if there's 1) a correlation between variance and read count, and 2) if there's a final bias in the samples chosen in each of the schemes toward samples with lower read counts. The second approach is probably a bit simpler but doesn't explicitly diagnose the issue in my current context. That is to test the impact of setting ever higher thresholds for read counts and testing how well the model performs with a given, small number of samples.